In [1]:
import os

import pandas as pd

In [15]:
#construccion ruta datos Bronce donde estan datos Bronce
bronce_path = os.path.join(
    os.curdir, "Data_Lake", "Bronce"
)

#construccion ruta datos Silver
silver_path = os.path.join(
    os.curdir, "Data_Lake", "Silver"
)

# Nombre del archivo parquet de entrada (donde datos originales)
bronce_file_name = "erp_bronce.parquet"

# Nombre del archivo parquet de salida (donde guardo datos procesados-limpios)
silver_file_name = "erp_silver.parquet"

# Ruta completa del archivo Bronce y Silver
bronce_file_path = os.path.join(
    bronce_path, bronce_file_name
)
silver_file_path = os.path.join(
    silver_path, silver_file_name
)

# lectura datos bronce (primeras filas)
bronce_erp_df = pd.read_parquet(bronce_file_path)
bronce_erp_df.head()


# Compruebo si la columna de id contiene nulos (si aperecieron)
bronce_erp_df[bronce_erp_df['RAZONSOCIAL'].isnull() == True]

#bronce_erp_df.info ()


# Borramos filas donde la columna es nula (ok) (eliminacion de 2 filas)
bronce_erp_df = bronce_erp_df.dropna(subset=["RAZONSOCIAL"])

bronce_erp_df[bronce_erp_df['RAZONSOCIAL'].isnull() == True]



#Aplico otro filtro para eliminar las filas si valores nulos en conjunto de columnas (identificacion empresas)
#puedo usar "any" para eliminar fila si algun valor nulo en alguna de las columnas
#bronce_erp_df es el df
resto_identificacion = ["SITIOWEB", "DIRECCION", "EMAIL", "TELEFONO"]
bronce_erp_df = bronce_erp_df.dropna(subset=resto_identificacion, how="all")



#ELIMINACION COLUMNAS INNECESARIAS

bronce_erp_df = bronce_erp_df.drop(columns=['TELEFONO2', 'FAX'])

#bronce_erp_df.info()

# Defino como voy a tratar los nulos en el resto de columnas (no trato empleo total ni rango facturacion)
#apunte: df.fillna(valor_o_diccionario), aqui vamos con el diccionario en la variable rellenar_valores
rellenar_valores = {
    "NOMBRECOMERCIAL": "No especificado",
    "DIRECCION": "No especificado",
    "SITIOWEB": "No disponible",
    "EMAIL": "No disponible",
    "TELEFONO": "No disponible",
    "PRODUCTOS ": "No especificado",
    "IMPORTA": "No",
    "EXPORTA": "No"
}

# Aplico la limpieza
bronce_erp_clean_df = bronce_erp_df.fillna(rellenar_valores)

bronce_erp_clean_df.info ()

<class 'pandas.core.frame.DataFrame'>
Index: 1949 entries, 0 to 1948
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   RAZONSOCIAL         1949 non-null   object        
 1   NOMBRECOMERCIAL     1949 non-null   object        
 2   DIRECCION           1949 non-null   object        
 3   SITIOWEB            1949 non-null   object        
 4   EMAIL               1949 non-null   object        
 5   TELEFONO            1949 non-null   object        
 6   PRODUCTOS           1944 non-null   object        
 7   EMPLEOTOTAL         204 non-null    float64       
 8   RANGOFACTURACION    1949 non-null   object        
 9   IMPORTA             1949 non-null   object        
 10  EXPORTA             1949 non-null   object        
 11  FECHAACTUALIZACION  1949 non-null   object        
 12  _BronzeTimestamp    1949 non-null   datetime64[us]
dtypes: datetime64[us](1), float64(1), object(11)
memory u